In [13]:
import biobb_structure_checking
import biobb_structure_checking.constants as cts
from biobb_structure_checking.structure_checking import StructureChecking
base_dir_path=biobb_structure_checking.__path__[0]
args = cts.set_defaults(base_dir_path,{'notebook':True})

## STEP 2

In [14]:
import argparse
import sys
import os
import math

from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.NACCESS import NACCESS_atomic
from Bio.PDB.NeighborSearch import NeighborSearch
from Bio.PDB.PDBIO import PDBIO, Select

**IMPORT PARAMETERS OF THE RESIDUE LIBRARY**

In [15]:
class ResiduesDataLib():
    def __init__(self, fname):
        self.residue_data = {}
        try:
            fh = open(fname, "r")
        except OSError:
            print("#ERROR while loading library file (", fname, ")")
            sys.exit(2)
        for line in fh:
            if line[0] == '#':
                continue
            data = line.split()
            r = Residue(data)
            self.residue_data[r.id] = r
        self.nres = len(self.residue_data)

    def get_params(self, resid, atid):
        atom_id = resid + ':' + atid
        if atom_id in self.residue_data:
            return self.residue_data[atom_id]
        else:
            print("WARNING: atom not found in library (", atom_id, ')')
            return None

Defining classes

In [16]:
class Residue():
    def __init__(self, data):
        self.id = data[0] + ':' + data[1]
        self.at_type = data[2]
        self.charge = float(data[3])
        
class AtomType():
    def __init__(self, data):
        self.id   = data[0]
        self.eps  = float(data[1])
        self.sig  = float(data[2])
        self.mass = float(data[3])
        self.fsrf = float(data[4])
        self.rvdw = self.sig * 0.5612
        
class VdwParamset(): #extracted from GELPI's github
    #parameters for the VdW
    def __init__ (self, file_name):
        self.at_types = {}
        try:
            fh = open(file_name, "r")
        except OSError:
            print ("#ERROR while loading parameter file (", file_name, ")")
            sys.exit(2)
        for line in fh:
            if line[0] == '#':
                continue
            data = line.split()
            self.at_types[data[0]] = AtomType(data)
        self.ntypes = len(self.at_types)
        fh.close()

**LOAD PARAMERTERS FROM THE FILES**

IMPORTANT: Change the pathway for your own

In [17]:
res_lib = ResiduesDataLib('/home/irene/Documents/BIOPYISICS/step 2/residue_library.txt')

ff_params = VdwParamset('/home/irene/Documents/BIOPYISICS/step 2/vdwprm')


In [18]:
# set the pdb_path and load the structure
pdb_path = '/home/irene/Documents/BIOPYISICS/step 2/6m0j_fixed.pdb'
# Setting the Bio.PDB.Parser object
parser = PDBParser(PERMISSIVE=1)
# Loading structure
st = parser.get_structure('st', pdb_path)

**SERCHING FOR ENERGIES TO PUT IN THE FORMULE**

In [19]:
def add_atom_parameters(st, res_lib, ff_params):
    ''' Adds parameters from libraries to atom .xtra field
        For not recognized atoms, issues a warning and put default parameters
    '''
    for at in st.get_atoms():
        resname = at.get_parent().get_resname()
        params = res_lib.get_params(resname, at.id)
        if not params:
            print("WARNING: residue/atom pair not in library ("+atom_id(at) + ')')
            at.xtra['atom_type'] = at.element
            at.xtra['charge'] = 0
        else:
            at.xtra['atom_type'] = params.at_type
            at.xtra['charge'] = params.charge
        at.xtra['vdw'] = ff_params.at_types[at.xtra['atom_type']]

CALL FUNCTION TO ENTER PARAMETERS
* res_lib and ff_ params --> parameters already imported above
* st --> structure of the protein, file imported from the path (above) (load parameters section)

In [20]:
add_atom_parameters(st, res_lib, ff_params)

los atoms que dan error son los que se ubican en la c terminal and n terminal
solucion: corregir en el mismo file los warning
* GLY OXT    O     -0.5722 --> i have to modified from GLYN 

In [21]:
def get_interface(st, dist):
    ''' Detects interface residues within a distance(dist)
        Assumes two chains, i.e. a unique interface set per chain.
    '''
    select_ats = []
    for at in st.get_atoms():
        # Skip Hydrogens to reduce time
        if at.element != 'H':
            select_ats.append(at)
    nbsearch = NeighborSearch(select_ats)
    interface = {}
    # Sets are more efficient than lists. Use sets when order is not relevant
    for ch in st[0]:
        interface[ch.id] = set()

    for at1, at2 in nbsearch.search_all(dist):
        #Only different chains
        res1 = at1.get_parent()
        ch1 = res1.get_parent()
        res2 = at2.get_parent()
        ch2 = res2.get_parent()
        if ch1 != ch2:
            interface[ch1.id].add(res1)
            interface[ch2.id].add(res2)
    return interface

In [22]:
get_interface(st, 3.5) # These are the residues of the interface

{'A': {<Residue GLN het=  resseq=24 icode= >,
  <Residue PHE het=  resseq=28 icode= >,
  <Residue ASP het=  resseq=30 icode= >,
  <Residue LYS het=  resseq=31 icode= >,
  <Residue HIE het=  resseq=34 icode= >,
  <Residue GLU het=  resseq=35 icode= >,
  <Residue GLU het=  resseq=37 icode= >,
  <Residue ASP het=  resseq=38 icode= >,
  <Residue TYR het=  resseq=41 icode= >,
  <Residue GLN het=  resseq=42 icode= >,
  <Residue TYR het=  resseq=83 icode= >,
  <Residue LYS het=  resseq=353 icode= >,
  <Residue GLY het=  resseq=354 icode= >,
  <Residue ASP het=  resseq=355 icode= >},
 'E': {<Residue LYS het=  resseq=417 icode= >,
  <Residue GLY het=  resseq=446 icode= >,
  <Residue TYR het=  resseq=449 icode= >,
  <Residue TYR het=  resseq=453 icode= >,
  <Residue ASN het=  resseq=487 icode= >,
  <Residue TYR het=  resseq=489 icode= >,
  <Residue GLN het=  resseq=493 icode= >,
  <Residue GLY het=  resseq=496 icode= >,
  <Residue GLN het=  resseq=498 icode= >,
  <Residue THR het=  resseq=500 ic

In [23]:
def MH_diel(r):
    '''Mehler-Solmajer dielectric'''
    return 86.9525 / (1 - 7.7839 * math.exp(-0.3153 * r)) - 8.5525


def elec_int(at1, at2, r):
    '''Electrostatic interaction energy between two atoms at r distance'''
    return 332.16 * at1.xtra['charge'] * at2.xtra['charge'] / MH_diel(r) / r


def vdw_int(at1, at2, r):
    '''Vdw interaction energy between two atoms'''
    eps12 = math.sqrt(at1.xtra['vdw'].eps * at2.xtra['vdw'].eps)
    sig12_2 = at1.xtra['vdw'].sig * at2.xtra['vdw'].sig
    return 4 * eps12 * (sig12_2**6/r**12 - sig12_2**3/r**6)


def calc_solvation(st, res):
    '''Solvation energy based on ASA for one residue'''
    solv = 0.
    for at in res.get_atoms():
        if 'EXP_NACCESS' not in at.xtra:
            continue
        s = float(at.xtra['EXP_NACCESS'])* at.xtra['vdw'].fsrf
        solv += s
    return solv

In [24]:
def calc_int_energies(st, res):
    '''Returns interaction energies (residue against other chains)
        for all atoms and for Ala atoms
    '''
    elec = 0.
    vdw = 0.

    for at1 in res.get_atoms():
        for at2 in st.get_atoms():
        # skip same chain atom pairs
            if at2.get_parent().get_parent() != res.get_parent():
                r = at1 - at2
                e = elec_int(at1, at2, r)
                elec += e
                e = vdw_int(at1, at2, r)
                vdw += e
    return elec, vdw

**CALLCULATION OF THE TOTAL ENERGY** (for all the residues)

IMPORTANT: 
- Change the NACCESS_BINARY pathway for your own
- Check if the pathway in naccess file (EXE_PATH) is correct

In [29]:
NACCESS_BINARY = '/home/irene/Documents/BIOPYISICS/step 2/Parameters_solvation/soft/NACCESS/naccess'
srf = NACCESS_atomic(st[0], naccess_binary = NACCESS_BINARY)
io = PDBIO()
st_chains = {}

# Using BioIO trick (see tutorial) to select chains
class SelectChain(Select):
    def __init__(self, chid):
        self.id = chid

    def accept_chain(self, chain):
        if chain.id == self.id:
            return 1
        else:
            return 0

for ch in st[0]:
    io.set_structure(st)
    io.save('tmp.pdb', SelectChain(ch.id))
    st_chains[ch.id] = parser.get_structure('stA', 'tmp.pdb')
    add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
    srfA = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)
os.remove('tmp.pdb')


## Initiatlize Energy aggregates
elec = {}
vdw = {}
solvAB = {}
solvA = {}

totalIntElec = 0.
totalIntVdw = 0.
totalSolv = 0.
totalSolvMon = {}

## We get the chains ids
chids = []
for ch in st[0]:
    chids.append(ch.id)
    totalSolvMon[ch.id] = 0

total = 0.

for ch in st[0]:
    for res in ch.get_residues():
        elec[res], vdw[res] = calc_int_energies(st[0], res)
        solvAB[res] = calc_solvation(st[0], res)
        solvA[res]  = calc_solvation(st_chains[ch.id],st_chains[ch.id][0][ch.id][res.id[1]])
        totalIntElec += elec[res]
        totalIntVdw += vdw[res]
        totalSolv += solvAB[res]
        totalSolvMon[ch.id] += solvA[res]
        total += elec[res] + vdw[res] + solvAB[res] - solvA[res]

print()
print('Total Elec Int. =', totalIntElec)
print('Total Vdw Int. =', totalIntVdw)
print('Total Solvation Complex =', totalSolv)
print('Total Solv ', chids[0], '=', totalSolvMon[chids[0]])
print('Total Solv ', chids[1], '=', totalSolvMon[chids[1]])
print('DG int =', total)

Exception: NACCESS did not execute or finish properly.

**CALCULATION OF THE TOTAL ENERGY** (for the interface residues)

IMPORTANT: Change the pathway for your own

In [31]:
io = PDBIO()
st_chains = {}
NACCESS_BINARY = '/home/irene/Documents/BIOPYISICS/step 2/Parameters_solvation/soft/NACCESS/naccess'
srf = NACCESS_atomic(st[0], naccess_binary = NACCESS_BINARY)


# Using BioIO trick (see tutorial) to select chains
class SelectChain(Select):
    def __init__(self, chid):
        self.id = chid

    def accept_chain(self, chain):
        if chain.id == self.id:
            return 1
        else:
            return 0

for ch in st[0]:
    io.set_structure(st)
    io.save('tmp.pdb', SelectChain(ch.id))
    st_chains[ch.id] = parser.get_structure('stA', 'tmp.pdb')
    add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
    srfA = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)
os.remove('tmp.pdb')

# Interface residues

max_dist = 3.5

if max_dist > 0.:
    interface = get_interface(st, max_dist)

# Initiatlize Energy aggregates

elec = {}
##elec_ala = {}

vdw = {}
##vdw_ala = {}

solvAB = {}
##solvAB_ala = {}

solvA = {}
##solvA_ala = {}

totalIntElec = 0.
totalIntVdw = 0.
totalSolv = 0.
totalSolvMon = {}

chids = []
for chain in st[0]:
    chids.append(chain.id)
    totalSolvMon[chain.id] = 0

total = 0.

for chain in st[0]:
    for res in chain.get_residues():
        if max_dist > 0 and res not in interface[chain.id]:
            continue
        elec[res], vdw[res] = calc_int_energies(st[0], res)
        solvAB[res] = calc_solvation(st[0], res)
        solvA[res] = calc_solvation(st_chains[chain.id],st_chains[chain.id][0][chain.id][res.id[1]])

        totalIntElec += elec[res]
        totalIntVdw += vdw[res]
        totalSolv += solvAB[res]
        totalSolvMon[chain.id] += solvA[res]

        total += elec[res] + vdw[res] + solvAB[res] - solvA[res]

print()
print('Total Electrostatic Int = ', totalIntElec)
print('Total Vdw Int = ', totalIntVdw)
print('Total Solvation Complex = ', totalSolv)
print('Total Solv', chids[0], '=', totalSolvMon[chids[0]])
print('Total Solv', chids[1], '=', totalSolvMon[chids[1]])
print('DGintAB-A-B = ', total)

Exception: NACCESS did not execute or finish properly.